In [28]:
import numpy as np
import scipy as sp
from scipy import stats as sps

In [1]:
def generate_data(N, P_L, beta_0_A, beta_0_Y, beta_A_Y):
    """
    1. outcome model: P(Y=1 | A, L) = sigmoid(beta_Y + beta_Y_given_A*A)
    2. propensity model: P(A=1 | L)
    """
    L_size = np.hstack([[N], np.array(P_L).shape])
    L = np.random.uniform(size=L_size) < P_L
    beta_A = [np.array(beta) for beta in beta_A]
    P_A = sps.logistic.cdf(
        beta_A[0] +
        np.einsum('ij,j', L, beta_A[1]) +
        np.einsum('ij,ik,jk', L, L, beta_A[2])
        )
    P_A = sps.logistic.cdf(
        beta_A[0] +
        np.einsum('ij,j', L, beta_A[1]) +
        np.einsum('ij,ik,jk', L, L, beta_A[2])
        )
    
    return {
        'L': L,
        'P_A': P_A,
    }

In [2]:
generate_data(10, [0.2, 0.8], [0, [1,-1], [[0, 1],[0, 0]]], 0)

TypeError: generate_data() missing 1 required positional argument: 'beta_Y_given_not_A'

In [33]:
np.pad([], 3).dot(4)

array([0., 0., 0., 0., 0., 0.])

In [16]:
np.random.uniform(size=np.hstack([[10], np.array([2,3]).shape])) < [0.8,0.1]

array([[ True, False],
       [ True, False],
       [False, False],
       [ True, False],
       [ True,  True],
       [False, False],
       [ True, False],
       [ True, False],
       [False, False],
       [ True, False]])

In [9]:
[10] + np.array([1,2]).shape

TypeError: can only concatenate list (not "tuple") to list

In [2]:
#N = 20 # number of patients
anxiety_driven_model = {
    'P_A': 0.3, # prevelance of baseline anxiety disorder
    'P_B_given_A': 0.5, # chance of benzodiazepine administration given baseline anxiety
    'P_B_given_not_A': 0.1, # chance of benzodiazepine administration given no baseline anxiety
    'P_D_given_A_B': 0.5, # chance of delirium with benzodiazepines and anxiety
    'P_D_given_A_not_B': 0.5, # chance of delirium with anxiety without benzodiazepines
    'P_D_given_B_not_A': 0.5, # chance of delirium with benzodiazepines and without anxiety
    'P_D_given_not_A_not_B': 0.2, # chance of delirium without benzodiazepines and without anxiety
}

In [3]:

def generate_data(N, 
        P_A, 
        P_B_given_A, 
        P_B_given_not_A, 
        P_D_given_A_B, 
        P_D_given_A_not_B, 
        P_D_given_B_not_A, 
        P_D_given_not_A_not_B, 
        ):
    r""" Generates a sample dataset with a given set of underlying probabilities.

    Parameters
    ----------
    N : int
        number of patients
    P_A : float
        prevelance of baseline anxiety disorder
    P_B_given_A : float
        chance of benzodiazepine administration given baseline anxiety
    P_B_given_not_A : float
        chance of benzodiazepine administration given no baseline anxiety
    P_D_given_A_B : float
        chance of delirium with benzodiazepines and anxiety
    P_D_given_A_not_B : float
        chance of delirium with anxiety without benzodiazepines
    P_D_given_B_not_A : float
        chance of delirium with benzodiazepines and without anxiety
    P_D_given_not_A_not_B : float
        chance of delirium without benzodiazepines and without anxiety
    """
    A = np.random.uniform(size=N) < P_A
    P_B = A * P_B_given_A + ~A * P_B_given_not_A
    B = np.random.uniform(size=N) < P_B
    P_D = (
        A * B * P_D_given_A_B +
        A * ~B * P_D_given_A_not_B +
        ~A * B * P_D_given_B_not_A +
        ~A * ~B * P_D_given_not_A_not_B
    )
    D = np.random.uniform(size=N) < P_D

    return {
        'P_A' : P_A * np.ones_like(P_B),
        'A': A,
        'P_B' : P_B,
        'B': B,
        'P_D': P_D,
        'D' : D
    }

In [4]:
# example of the data generated
generate_data(10, 0.3, 0.5, 0.1, 0.6, 0.2, 0.4, 0.1)

{'P_A': array([0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3]),
 'A': array([False, False,  True, False, False, False, False,  True, False,
        False]),
 'P_B': array([0.1, 0.1, 0.5, 0.1, 0.1, 0.1, 0.1, 0.5, 0.1, 0.1]),
 'B': array([False, False,  True, False,  True, False, False,  True, False,
        False]),
 'P_D': array([0.1, 0.1, 0.6, 0.1, 0.4, 0.1, 0.1, 0.6, 0.1, 0.1]),
 'D': array([False, False, False, False, False, False, False,  True, False,
        False])}

In [5]:
def summarize_model_results(results):
    print(
        f"observed anxiety prevelance: {np.mean(results['A'])}\n" +
        f"observed delirium prevelance: {np.mean(results['D'])}\n"
    )

In [6]:
N = 1000000

In [7]:
# Example of an additive model
additive_model = {
    'P_A': 0.3,
    'P_B_given_A': 0.5,
    'P_B_given_not_A': 0.1,
    'P_D_given_A_B': 0.7,
    'P_D_given_A_not_B': 0.3,
    'P_D_given_B_not_A': 0.4,
    'P_D_given_not_A_not_B': 0.1,
}
additive_model_results = generate_data(N, **additive_model)
summarize_model_results(additive_model_results)

observed anxiety prevelance: 0.299476
observed delirium prevelance: 0.240423



In [8]:
# Example of a model where anxiety replaces benzodiazepine risk
anxiety_driven_model = {
    'P_A': 0.3,
    'P_B_given_A': 0.5,
    'P_B_given_not_A': 0.1,
    'P_D_given_A_B': 0.5,
    'P_D_given_A_not_B': 0.5,
    'P_D_given_B_not_A': 0.4,
    'P_D_given_not_A_not_B': 0.1,
}
anxiety_driven_model_results = generate_data(N, **anxiety_driven_model)
summarize_model_results(anxiety_driven_model_results)

observed anxiety prevelance: 0.299571
observed delirium prevelance: 0.241139



In [9]:
# Example of a model where benzodiazepine mediate all risk
benzo_driven_model = {
    'P_A': 0.3,
    'P_B_given_A': 0.5,
    'P_B_given_not_A': 0.1,
    'P_D_given_A_B': 0.7,
    'P_D_given_A_not_B': 0.11,
    'P_D_given_B_not_A': 0.7,
    'P_D_given_not_A_not_B': 0.11,
}
benzo_driven_model_results = generate_data(N, **benzo_driven_model)
summarize_model_results(benzo_driven_model_results)

observed anxiety prevelance: 0.300216
observed delirium prevelance: 0.240004



In [14]:
# Example of a model where benzodiazepine are protective with anxiety
benzo_protective_model = {
    'P_A': 0.3,
    'P_B_given_A': 0.5,
    'P_B_given_not_A': 0.1,
    'P_D_given_A_B': 0.7,
    'P_D_given_A_not_B': 0.3,
    'P_D_given_B_not_A': 0.4,
    'P_D_given_not_A_not_B': 0.1,
}
benzo_protective_model_results = generate_data(N, **benzo_protective_model)
summarize_model_results(benzo_protective_model_results)

observed anxiety prevelance: 0.300505
observed delirium prevelance: 0.240846



In [15]:
# Example of a model where benzodiazepine have no effect
benzo_neutral_model = {
    'P_A': 0.3,
    'P_B_given_A': 0.5,
    'P_B_given_not_A': 0.1,
    'P_D_given_A_B': 0.6,
    'P_D_given_A_not_B': 0.6,
    'P_D_given_B_not_A': 0.1,
    'P_D_given_not_A_not_B': 0.1,
}
benzo_neutral_model_results = generate_data(N, **benzo_neutral_model)
summarize_model_results(benzo_neutral_model_results)

observed anxiety prevelance: 0.299201
observed delirium prevelance: 0.249889



Before next meeting:

 - create parametric model for generating results (e.g. logistic regression)
 - add at least one binary covariate (e.g. age > 70) $L$

Next time:

- First look at total effect from Anxiety to Delirium
    - Outcome regression / standardization / G-formula